In [ ]:
import os

from src.aggregates import obr_per_subtype_per90min
from src.load_data import load_matches


Total off-ball runs by subtype + untargeted share
% of run types by phase of play
Two radar plots: teams and players

and then sth to move to midfielders
and then the DEEP ANALYSIS and metrics creation

In [ ]:
# load matches
matches_json_path = os.path.join(os.getcwd(), "src/data/matches.json")
all_tracking, dynamic_events_all, all_metadata = load_matches(matches_json_path)

# compute off-ball event stats per subtype per90min
obr_stats_per90min = obr_per_subtype_per90min(all_tracking, dynamic_events_all)

#print(obr_stats_per90min)

c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (75,77,184,264) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)
c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (276) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)


Total played: 97.4 minutes
Total played: 99.8 minutes
Total played: 94.7 minutes
Total played: 98.8 minutes
Total played: 98.7 minutes
Total played: 96.0 minutes
Total played: 98.7 minutes
Total played: 102.1 minutes
Total played: 100.8 minutes
Total played: 98.4 minutes
   event_id  index  match_id  frame_start  frame_end  frame_physical_start  \
1       1_0      1   2017461         2523       2539                   NaN   
6       1_1      6   2017461         2529       2544                   NaN   
9       1_2      9   2017461         2541       2550                   NaN   
19      1_3     19   2017461         2629       2661                   NaN   
20      1_4     20   2017461         2635       2658                   NaN   

   time_start time_end  minute_start  second_start  ...  \
1     00:01.3  00:02.9             0             1  ...   
6     00:01.9  00:03.4             0             1  ...   
9     00:03.1  00:04.0             0             3  ...   
19    00:11.9  00:15.1 

KeyError: 'targeted'